In [1]:
from tqdm.notebook import tqdm
import numpy as np
import pickle
import scipy.sparse as sp
import cupy as cp

In [2]:
with open('vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)
with open('word2ind.pkl', 'rb') as f1:
    word2ind = pickle.load(f1)
with open('ind2word.pkl', 'rb') as f2:
    ind2word = pickle.load(f2)

In [3]:
x = sp.load_npz('x_cbow.npz')
y = sp.load_npz('y_cbow.npz')

In [4]:
cp.random.seed(1)

In [5]:
W0=cp.random.uniform(low=-0.8 ,high=0.8,size=(len(vocabulary),200))
W1=cp.random.uniform(low=-0.8 ,high=0.8,size=(200,len(vocabulary)))

In [6]:
def softmax(z):
    e_z = cp.exp(z)
    sum_e_z = cp.sum(e_z,axis=0,keepdims=True)
    return e_z/sum_e_z

In [7]:
def forward(W0,W1,x):
    h=cp.dot(W0.T,x)
    v=cp.dot(W1.T,h)
    return h,softmax(v)

In [8]:
def backprop(W0,W1,x,y_true,alpha):
    h,y_pred=forward(W0,W1,x)
    y_pred=cp.maximum(1e-8,y_pred)
    loss = - cp.sum(y_true*np.log(y_pred))/y_true.shape[1]
    l1 = cp.dot(W1 ,(y_pred - y_true))
    dW1 = cp.dot(h,(y_pred - y_true).T)
    dW0 = cp.dot(x,l1.T)
    W1 -= alpha * dW1
    W0 -= alpha * dW0
    return W0,W1,loss

In [9]:
def minibatch(W0,W1,x,y,leng,epochs):
    W0x,W1x=W0,W1
    e=0
    while e<=epochs:
        info=""
        batchsize=100
        i=0
        while batchsize+i <= leng:    
            W0x,W1x,loss=backprop(W0x,W1x,cp.asarray(x[:,i:i+batchsize].toarray()),cp.asarray(y[:,i:i+batchsize].toarray()),0.03)
            if (i+batchsize)% 50000==0:
                info+=f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}\n"
                print(f"loss at epoch {e} with batch {i} - {i+batchsize} is {loss}")
          
            if i+batchsize == leng:
                info+=f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}\n"
                print(f"returned to batch of epoch {e} with loss at {i} - {i+batchsize} is {loss}")

                if not np.isnan(W0x[1,2]):
                    print(f"content w0,w1 saved for epoch {e} at {i+batchsize}")
                    np.save(f'W0epoch{e}.npy',W0x)
                    np.save(f'W1epoch{e}.npy',W1x)
                    with open('loss.txt', 'a') as file:
                          file.write(info)

            if i+batchsize == 659400:
              batchsize=(leng-(batchsize+i))
          
            i+=100
      
        e+=1
    

In [10]:
leng=x.shape[1]

In [ ]:
minibatch(W0,W1,x,y,leng,1)